In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
#from vis_utils import *
import random;
import math;
import cv2

In [2]:
# List images and label them

from random import shuffle
import glob
shuffle_data = True  # shuffle the addresses before saving
hdf5_path = 'E:/Object_Detection/HD5/dataset155.hdf5'  # address to where you want to save the hdf5 file
cat_dog_train_path = 'E:/Object_Detection/HD5/Cats/*.jpg'
# read addresses and labels from the 'train' folder
addrs = glob.glob(cat_dog_train_path)
labels = [0 if 'cat' in addr else 1 for addr in addrs]  # 0 = Cat, 1 = Dog
# to shuffle data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)
    
# Divide the Data into 60% train, 20% validation, and 20% test
train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]
val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]
test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

In [3]:
import numpy as np
import h5py
data_order = 'th'  # 'th' for Theano, 'tf' for Tensorflow
# check the order of data and chose proper data shape to save images
if data_order == 'th':
    train_shape = (len(train_addrs), 3, 224, 224)
    val_shape = (len(val_addrs), 3, 224, 224)
    test_shape = (len(test_addrs), 3, 224, 224)
elif data_order == 'tf':
    train_shape = (len(train_addrs), 224, 224, 3)
    val_shape = (len(val_addrs), 224, 224, 3)
    test_shape = (len(test_addrs), 224, 224, 3)
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_img", train_shape, np.int8)
hdf5_file.create_dataset("val_img", val_shape, np.int8)
hdf5_file.create_dataset("test_img", test_shape, np.int8)
hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
hdf5_file.create_dataset("train_labels", (len(train_addrs),), np.int8)
hdf5_file["train_labels"][...] = train_labels
hdf5_file.create_dataset("val_labels", (len(val_addrs),), np.int8)
hdf5_file["val_labels"][...] = val_labels
hdf5_file.create_dataset("test_labels", (len(test_addrs),), np.int8)
hdf5_file["test_labels"][...] = test_labels

In [4]:
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in range(len(train_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image and calculate the mean so far
    hdf5_file["train_img"][i, ...] = img[None]
    mean += img / float(len(train_labels))
# loop over validation addresses
for i in range(len(val_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print ('Validation data: {}/{}'.format(i, len(val_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = val_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image
    hdf5_file["val_img"][i, ...] = img[None]
# loop over test addresses
for i in range(len(test_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image
    hdf5_file["test_img"][i, ...] = img[None]
# save the mean and close the hdf5 file
hdf5_file["train_mean"][...] = mean
hdf5_file.close()

In [5]:
import h5py
import numpy as np
hdf5_path = 'E:/Object_Detection/HD5/dataset155.hdf5'
subtract_mean = False
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")
# subtract the training mean
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
# Total number of samples
data_num = hdf5_file["train_img"].shape[0]

In [6]:
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

batch_size = 10
nb_class = 2
# create list of batches to shuffle the data
batches_list = list(range(int(ceil(float(data_num) / batch_size))))
shuffle(batches_list)
# loop over batches
for n, i in enumerate(batches_list):
    i_s = i * batch_size  # index of the first image in this batch
    i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
    # read batch images and remove training mean
    images = hdf5_file["train_img"][i_s:i_e, ...]
    if subtract_mean:
        images -= mm
    # read labels and convert to one hot encoding
    labels = hdf5_file["train_labels"][i_s:i_e]
    labels_one_hot = np.zeros((batch_size, nb_class))
    labels_one_hot[np.arange(batch_size), labels] = 1
    print (n+1, '/', len(batches_list))
    print (labels[0], labels_one_hot[0, :])
#     plt.imshow(images[0])
#     plt.show()
    if n == 5:  # break after 5 batches
        break
hdf5_file.close()

1 / 55
0 [1. 0.]
2 / 55
0 [1. 0.]
3 / 55
1 [0. 1.]
4 / 55
1 [0. 1.]
5 / 55
1 [0. 1.]
6 / 55
1 [0. 1.]


In [7]:
import h5py    # HDF5 support
import torch
import numpy as np

fileName = 'E:/Object_Detection/HD5/dataset155.hdf5'
file = h5py.File(fileName)
xtrainT = torch.from_numpy(np.array(file['train_img'], dtype=np.float32)).float()
ytrainT = torch.from_numpy(np.array(file['train_labels'], dtype=np.float32)).float()
# for item in f.attrs.keys():
#     print(item + ":", f.attrs[item])

In [8]:
print(xtrainT.shape,ytrainT.shape)

torch.Size([541, 3, 224, 224]) torch.Size([541])


In [9]:
fileName = 'E:/Object_Detection/HD5/dataset155.hdf5'
file = h5py.File(fileName)
xtestT = torch.from_numpy(np.array(file['test_img'], dtype=np.float32)).float()
ytextT = torch.from_numpy(np.array(file['test_labels'], dtype=np.float32)).float()

In [10]:
print(xtestT.shape,ytextT.shape)

torch.Size([181, 3, 224, 224]) torch.Size([181])


In [ ]:
pwd

In [11]:
num_epochs = 3;
batch_size = 10;
learning_rate = 0.001;

In [12]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(xtrainT, ytrainT)
test = data_utils.TensorDataset(xtestT, ytextT)

In [13]:
train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size,
                                            shuffle=False);
test_loader = torch.utils.data.DataLoader(dataset=test,
                                          batch_size=batch_size,
                                          shuffle=False);

In [40]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = 'D:/all_Bringal'
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader('E:/Object_Detection/HD5/dataset155.hdf5', batch_size=10,
                                             shuffle=False, num_workers=4)
              for x in ['train_img', 'val_img', 'test_img']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
# class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
inputs, classes = next(iter(dataloaders['train_img']))
print(classes)

ValueError: too many values to unpack (expected 4)

In [15]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 7, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, 2)
        self.conv3 = nn.Conv2d(16, 20, 3, 1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(20 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(2, 2))
  (conv3): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=500, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [16]:
import torch.nn.functional as F
input = torch.randn(1, 3, 224, 224)
out = net(input)
print(out.shape)

torch.Size([1, 2])


In [17]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [18]:
#losses = [];
for epoch in range(5):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        images = Variable(images.float())
        labels = Variable(labels.float())
        #inputs, labels = data
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2 == 1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2))
            running_loss = 0.0

print('Finished Training')

RuntimeError: The size of tensor a (2) must match the size of tensor b (10) at non-singleton dimension 1

In [35]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=3):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train_img', 'val_img']:
            if phase == 'train_img':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train_img'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train_img':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(),'weight_Thdf.pth')
    return model

In [36]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [38]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=6)

Epoch 0/5
----------


ValueError: too many values to unpack (expected 2)

In [63]:
net.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.float())
    labels = Variable(labels.long())
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('Test Accuracy of the model on the 1000 test images: %.4f %%' % (100 * correct / total))

Test Accuracy of the model on the 1000 test images: 33.0000 %


In [ ]:
torch.cuda.current_device()

In [ ]:
alexnet = models.alexnet(pretrained=True)

In [ ]:
alexnet= nn.Sequential(*list(alexnet.features))
alexnet

In [ ]:
input = Variable(torch.randn(1, 3, 228, 304))
out = alexnet(input)
print(out.shape)

In [ ]:
def InitializeWeights(mod):
    for m in mod.modules():
        if isinstance(m,nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            #print m.weight.size(), m.out_channels, m.in_channels
            m.weight.data.normal_(0,sqrt(2./n))
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
    return mod

In [ ]:
conv1 = nn.Sequential(nn.BatchNorm2d(256),nn.ReLU(),nn.Conv2d(256,1024,1))
conv1 = InitializeWeights(conv1)
conv2 = nn.Sequential(nn.BatchNorm2d(1024),nn.ReLU(),nn.Conv2d(1024,128,2))
conv2 = InitializeWeights(conv2)
conv3 = nn.Sequential(nn.BatchNorm2d(128),nn.ReLU(),nn.Conv2d(128,16,1))
conv3 = InitializeWeights(conv3)
norm1 = nn.BatchNorm2d(16)
norm1 = InitializeWeights(norm1)
fc1= nn.Sequential(nn.Linear(12288, 2000))
fc2= nn.Sequential(nn.Linear(2000, 2))

In [ ]:
class MyModel4(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel4, self).__init__()
        self.pretrained_model = nn.Sequential(alexnet)
#         self.conv1 = conv1
#         self.conv2 = conv2
#         self.conv3 = conv3
#         self.norm1 = norm1
        self.fc1 = fc1
        self.fc2 = fc2
    
    
   
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
    def forward(self, x):
        x = self.pretrained_model(x)
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = self.norm1(x)
        #print(x.size())
        x = x.view(-1, self.num_flat_features(x))
#         #print(x.size())
#         #x = self.conv4(x)
        x = self.fc1(x)
        x = F.softmax(self.fc2(x))
        return x
#print(net)

In [ ]:
net = MyModel4(alexnet)

In [ ]:
net

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

In [ ]:
sum1 = 0
        
print("Number of layers ---> ",len(list(net.parameters())))
for params in net.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

In [ ]:
input = torch.randn(1, 3, 228, 304)
out = net(input)
print(out)

In [ ]:
file = h5py.File('TrainData_3800.h5')
xtrainT = torch.from_numpy(np.array(file['xtrain'],dtype=np.float32)).float()
ytrainT = torch.from_numpy(np.array(file['ytrain'],dtype=np.float32)).float()

In [ ]:
print(xtrainT.size(), ytrainT.size())

In [ ]:
print(ytrainT[1])

In [ ]:
print(xtrainT[1].size())